# Practice notebook for regression analysis with dependent data in NHANES

This notebook will give you the opportunity to perform some analyses
using the regression methods for dependent data that we are focusing
on in this week of the course.

Enter the code in response to each question in the boxes labeled "enter your code here".
Then enter your responses to the questions in the boxes labeled "Type
Markdown and Latex".

This notebook is based on the NHANES case study notebook for
regression with dependent data.  Most of the code that you will need
to write below is very similar to code that appears in the case study
notebook.  You will need to edit code from that notebook in small ways
to adapt it to the prompts below.

To get started, we will use the same module imports and read the data
in the same way as we did in the case study:

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels.api as sm
import numpy as np

url = "nhanes_2015_2016.csv"
da = pd.read_csv(url)

# Drop unused columns, drop rows with any missing values.
vars = ["BPXSY1", "RIDAGEYR", "RIAGENDR", "RIDRETH1", "DMDEDUC2", "BMXBMI",
        "SMQ020", "SDMVSTRA", "SDMVPSU"]
da = da[vars].dropna()

# This is the grouping variable
da["group"] = 10*da.SDMVSTRA + da.SDMVPSU

## Question 1:

Build a marginal linear model using GEE for the first measurement of diastolic blood pressure (`BPXDI1`), accounting for the grouping variable defined above.  This initial model should have no covariates, and will be used to assess the ICC of this blood pressure measure.

In [18]:
# enter your code here

import statsmodels.api as sm
import statsmodels.genmod.cov_struct as cov_struct
da["group"] = 10*da.SDMVSTRA + da.SDMVPSU
qw=sm.GEE.from_formula('BPXDI1 ~ 1', groups='group', data=da)
qwer=qw.fit()
print(qwer.summary())


                               GEE Regression Results                              
Dep. Variable:                      BPXDI1   No. Observations:                 5401
Model:                                 GEE   No. clusters:                       30
Method:                        Generalized   Min. cluster size:                 111
                      Estimating Equations   Max. cluster size:                 234
Family:                           Gaussian   Mean cluster size:               180.0
Dependence structure:         Independence   Num. iterations:                     2
Date:                     Sat, 09 Dec 2023   Scale:                         165.935
Covariance type:                    robust   Time:                         17:12:34
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     69.5164      0.429    162.223      0.000      68.676      70.356
Skew:  

__Q1a.__ What is the ICC for diastolic blood pressure?  What can you
  conclude by comparing it to the ICC for systolic blood pressure?

# Q1a. The ICC for diastolic blood pressure is the scale parameter. Comparing it to systolic blood pressure's ICC can provide insights into the relative strength of the clustering effect.


## Question 2:

Take your model from question 1, and add gender, age, and BMI to it as covariates.

In [15]:
# enter your code here
import statsmodels.api as sm 
import statsmodels.genmod.cov_struct as cov_struct 
qw=sm.GEE.from_formula('BPXDI1 ~ RIDAGEYR + RIAGENDR + BMXBMI',groups='group',data=da)
qwer=qw.fit()
print(qwer.summary())


# Assuming you have already fitted the GEE model with covariates
var_between_groups_covariates = qwer.scale
total_variance_covariates = var_between_groups_covariates + qwer.scale

# Calculate ICC for the model with covariates
icc_covariates = var_between_groups_covariates / total_variance_covariates

# Print ICC for the model with covariates
print("ICC for Model with Covariates:", icc_covariates)



                               GEE Regression Results                              
Dep. Variable:                      BPXDI1   No. Observations:                 5347
Model:                                 GEE   No. clusters:                       30
Method:                        Generalized   Min. cluster size:                 110
                      Estimating Equations   Max. cluster size:                 230
Family:                           Gaussian   Mean cluster size:               178.2
Dependence structure:         Independence   Num. iterations:                     2
Date:                     Sat, 09 Dec 2023   Scale:                         160.602
Covariance type:                    robust   Time:                         16:24:47
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     68.5439      1.280     53.563      0.000      66.036      71.052
RIDAGEY

__Q2a.__ What is the ICC for this model?  What can you conclude by comparing it to the ICC for the model that you fit in question 1?

# Q2a. Comparing ICCs helps assess the impact of covariates on the clustering effect. A decrease in ICC suggests that covariates explain some between-group variability.


## Question 3:

Split the data into separate datasets for females and for males and fit two separate marginal linear models for diastolic blood pressure, one only for females, and one only for males.

In [9]:
# enter your code here
import statsmodels.api as sm
import statsmodels.genmod.cov_struct as cov_struct
qw=da['RIAGENDR'].values==2
qwer=da[qw]
zx=sm.GEE.from_formula('BPXDI1 ~ RIDAGEYR + RIAGENDR + BMXBMI', groups='group',data=qwer)
zxc=zx.fit()
print('for females:')
print(zxc.summary())
print('for males:')
ml=da['RIAGENDR'].values==1
mn=da[ml]
qw=sm.GEE.from_formula('BPXDI1 ~ RIDAGEYR + RIAGENDR + BMXBMI', groups='group',data=mn)
mnb=qw.fit()
print(mnb.summary())




for females:
                               GEE Regression Results                              
Dep. Variable:                      BPXDI1   No. Observations:                 2757
Model:                                 GEE   No. clusters:                       30
Method:                        Generalized   Min. cluster size:                  63
                      Estimating Equations   Max. cluster size:                 125
Family:                           Gaussian   Mean cluster size:                91.9
Dependence structure:         Independence   Num. iterations:                     2
Date:                     Sat, 09 Dec 2023   Scale:                         146.023
Covariance type:                    robust   Time:                         16:10:54
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     12.7812      0.303     42.208      0.000      12.188      1

__Q3a.__ What do you learn by comparing these two fitted models?

# Q3a. Comparing the fitted models for females and males helps assess whether gender-specific differences exist in the clustering effect on diastolic blood pressure.


For Females:


NameError: name 'result_female_q3' is not defined

## Question 4:

Using the entire data set, fit a multilevel model for diastolic blood pressure, predicted by age, gender, BMI, and educational attainment.  Include a random intercept for groups.

In [19]:
# enter your code here
qw=sm.GEE.from_formula('BPXDI1 ~ RIDAGEYR + RIAGENDR + BMXBMI', groups='group',data=da)
qwer=qw.fit()
print(qwer.summary())



                               GEE Regression Results                              
Dep. Variable:                      BPXDI1   No. Observations:                 5347
Model:                                 GEE   No. clusters:                       30
Method:                        Generalized   Min. cluster size:                 110
                      Estimating Equations   Max. cluster size:                 230
Family:                           Gaussian   Mean cluster size:               178.2
Dependence structure:         Independence   Num. iterations:                     2
Date:                     Sat, 09 Dec 2023   Scale:                         160.602
Covariance type:                    robust   Time:                         17:19:40
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     68.5439      1.280     53.563      0.000      66.036      71.052
RIDAGEY

__Q4a.__ How would you describe the strength of the clustering in this analysis?

# Q4a. The strength of clustering can be assessed by examining the variance of the random intercept for groups. A larger variance indicates stronger clustering.


__Q4b:__ Include a random intercept for age, and describe your findings.

# Q4b. Including a random intercept for age helps assess whether age-related variability contributes to the overall clustering effect.
